In [165]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix 
import scipy.io as scio
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

In [145]:
class Dataset:
    
    def __init__(self,name,form):
        
        self.name=name
        self.form=form
        
    def load(self):
        
        self.data_dict=scio.loadmat(self.name+self.form)
        self.data=self.data_dict[self.name]

In [146]:
bots=Dataset('Botswana','.mat')
bots.load()
data=bots.data
shp=np.shape(data)
row=shp[0]
col=shp[1]
bands=shp[2]
    
X=data.reshape(row*col,bands)
    
#Obtaining data in desired form
bots_gt=Dataset('Botswana_gt','.mat')
bots_gt.load()
y=bots_gt.data.reshape(row*col,1)
ind=np.where(y[:,0]!=0)
X=X[ind]
y = y[y!=0]

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
sc = StandardScaler()
sc.fit_transform(X_train)
Xn_train = sc.transform(X_train)
Xn_test = sc.transform(X_test)

pca = PCA(n_components=5)
pca.fit_transform(Xn_train)
Xpc_train = pca.transform(Xn_train)
Xpc_test = pca.transform(Xn_test)

C:\Users\Ankit\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [171]:
mu=dict()
icov=dict()
det=dict()
start_time = time.time()
for cls in np.unique(y_train):
    ind=np.where(y_train==cls)
    X_cls=Xpc_train[ind]
    mu[cls]=np.mean(X_cls,0)
    cov=np.cov(X_cls.T)
    det[cls]=np.linalg.det(cov)
    icov[cls] = np.linalg.inv(cov)
print("Training time is %s seconds ---" % (time.time() - start_time))

Training time is 0.1169278621673584 seconds ---


In [172]:
def find_class(test,mu,det,icov):
    tot_cls=len(det)
    pmax=-1000000000000000
    for i in list(range(1,15,1)):
        test_s=test-mu[i]
        logp=-np.log(det[i])-np.matmul(np.matmul(test_s,icov[i]),test_s.T)
    
        if(logp>pmax):
            pmax=logp
            y_pred=i
    return y_pred
    

In [173]:
y_pred=np.zeros(len((y_test)))
i=0
start_time = time.time()
for test in Xpc_test:
    y_pred[i]=find_class(test,mu,det,icov)
    i=i+1

print("Testing time is %s seconds ---" % (time.time() - start_time))

Testing time is 0.2168745994567871 seconds ---


In [174]:
acc=accuracy_score(y_test, y_pred)
print("Accuracy: %0.2f" % acc)
kappa_score = cohen_kappa_score(y_test,y_pred)
print("Kappa coefficient: %0.2f" % kappa_score)
print(confusion_matrix(y_test,y_pred))

Accuracy: 0.93
Kappa coefficient: 0.92
[[89  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0 87  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0 67  0  0  2  0  0  0  0  0  0  0]
 [ 0  4  0  1 61  8  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  8 79  0  0 10  0  0  0  0  0]
 [ 0  0  1  0  1  0 76  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 62  0  1  0  0  0  0]
 [ 0  0  0  0  0 14  0  0 96  2  0  0  3  0]
 [ 0  0  0  0  0  0  0  1  0 87  7  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  6 89  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 53  0  0]
 [ 0  0  0  0  0  1  0  0  0  1  0  5 87  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 25]]


In [143]:
det

{1: 3.99005990162426e-14,
 2: 1.9554133734685549e-11,
 3: 7.304961553738108e-12,
 4: 8.9330301792467667e-12,
 5: 7.1665826983255398e-09,
 6: 2.3494464286805212e-10,
 7: 4.8148054861725505e-10,
 8: 3.4517586735023008e-11,
 9: 2.7809496982380274e-11,
 10: 3.7070519725895129e-11,
 11: 3.2553565618853888e-09,
 12: 1.6356885076040704e-11,
 13: 1.0702361727177983e-10,
 14: 8.2435489896996506e-08}

In [151]:
det

{1: 0.0017412443866393255,
 2: 0.032848049487646795,
 3: 0.019447831158058711,
 4: 0.0228586509756332,
 5: 0.85549588939860599,
 6: 0.14508824572975557,
 7: 3.329763986513762,
 8: 0.10728924912949185,
 9: 0.053426005036556416,
 10: 0.036976737989452085,
 11: 0.42389957877867329,
 12: 0.029773302398706101,
 13: 0.13934707489277223,
 14: 64.163016920444363}

In [80]:
test

array([-1.96188079, -7.15473563,  1.88305575])